In [1]:
import tensorflow as tf
from keras import models, layers
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [2]:
IMAGE_SIZE = 224
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS=25

In [3]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = datagen.flow_from_directory(
    r"D:\Deep Learning & ML\pneumonia_detection\training\chest_x_ray_aug\train",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

Found 12230 images belonging to 2 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        r"D:\Deep Learning & ML\pneumonia_detection\training\chest_x_ray_aug\valid",
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical')

Found 1747 images belonging to 2 classes.


In [6]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes=2

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])
model.build(input_shape=input_shape)

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 64)        36928     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 64)        0

In [8]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [9]:
model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator
)

Epoch 1/25


383/383 [==============================] - 296s 767ms/step - loss: 0.5931 - accuracy: 0.7292 - val_loss: 0.5875 - val_accuracy: 0.7281
Epoch 2/25
383/383 [==============================] - 299s 780ms/step - loss: 0.4605 - accuracy: 0.7833 - val_loss: 0.2633 - val_accuracy: 0.8958
Epoch 3/25
383/383 [==============================] - 309s 806ms/step - loss: 0.2982 - accuracy: 0.8745 - val_loss: 0.1809 - val_accuracy: 0.9428
Epoch 4/25
383/383 [==============================] - 303s 791ms/step - loss: 0.2506 - accuracy: 0.8986 - val_loss: 0.1448 - val_accuracy: 0.9531
Epoch 5/25
383/383 [==============================] - 377s 985ms/step - loss: 0.2384 - accuracy: 0.9051 - val_loss: 0.1554 - val_accuracy: 0.9416
Epoch 6/25
383/383 [==============================] - 514s 1s/step - loss: 0.2086 - accuracy: 0.9187 - val_loss: 0.4080 - val_accuracy: 0.8071
Epoch 7/25
383/383 [==============================] - 513s 1s/step - loss: 0.2080 - accuracy: 0.9192 - val_loss: 0.1241 - val

In [10]:
model_version=2
model.save(f"D:\Deep Learning & ML\pneumonia_detection\model_updated\{model_version}")

INFO:tensorflow:Assets written to: D:\Deep Learning & ML\pneumonia_detection\model_updated\2\assets


INFO:tensorflow:Assets written to: D:\Deep Learning & ML\pneumonia_detection\model_updated\2\assets


In [5]:
from keras import models, layers
MODEL = tf.keras.models.load_model(r"D:\Deep Learning & ML\pneumonia_detection\model_cnn_updated\2")

from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
import numpy as np
# Get true labels and predicted labels for validation data
val_data, val_labels = validation_generator.next()
val_pred_probs = MODEL.predict(val_data)
val_pred_labels = np.argmax(val_pred_probs, axis=1)  # Convert predicted probabilities to labels

# Calculate accuracy
accuracy = accuracy_score(np.argmax(val_labels, axis=1), val_pred_labels)

# Calculate precision
precision = precision_score(np.argmax(val_labels, axis=1), val_pred_labels)

# Calculate recall
recall = recall_score(np.argmax(val_labels, axis=1), val_pred_labels)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

# You can also print a classification report which includes precision, recall, and F1-score for each class
print(classification_report(np.argmax(val_labels, axis=1), val_pred_labels))




1/1 [==============================] - 1s 621ms/step
Accuracy: 0.96875
Precision: 1.0
Recall: 0.9523809523809523
              precision    recall  f1-score   support

           0       0.92      1.00      0.96        11
           1       1.00      0.95      0.98        21

    accuracy                           0.97        32
   macro avg       0.96      0.98      0.97        32
weighted avg       0.97      0.97      0.97        32

